### Tool Calling Agent

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=gemini_api_key
)

In [3]:
from langchain_core.messages import HumanMessage
llm.invoke([HumanMessage("부산은 지금 몇시야?")])

AIMessage(content='부산은 지금 **2024년 5월 15일 수요일 오후 10시 58분** 입니다. (한국 시간 기준)', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--e470ac37-22b0-4044-9e1c-fbbf021f30a1-0', usage_metadata={'input_tokens': 8, 'output_tokens': 210, 'total_tokens': 218, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 175}})

In [9]:
from datetime import datetime
from zoneinfo import ZoneInfo

seoul_tz = ZoneInfo("Asia/Seoul")
newyork_tz = ZoneInfo("America/New_York")

print(datetime.now(tz=seoul_tz))
print(datetime.now(tz=newyork_tz))

2025-09-13 17:49:14.670565+09:00
2025-09-13 04:49:14.671049-04:00


In [11]:
from langchain_core.tools import tool

@tool
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수 """

    target_tz = ZoneInfo(timezone)
    now = datetime.now(tz=target_tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time=f"{timezone} ({location}) 현재시각 {now}"

    return location_and_local_time


In [13]:
# create_tool_calling_agent: 도구 호출 기능이 있는 에이전트를 생성하는 함수
# AgentExecutor: 생성된 에이전트의 실행을 담당하는 클래스
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# 도구들을 tools 리스트에 추가
tools = [get_current_time,]

# 에이전트 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# 에이전트 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 초기화
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # verbose 키워드는 AgentExecutor의 실행 과정을 자세하게 출력하도록 설정하는 키워드

In [14]:

# 메시지 형식 변경 (agent_executor에 맞게)
messages = [
    HumanMessage("부산은 지금 몇시야?"),
]

# agent_executor를 사용하여 실행
response = agent_executor.invoke({"input": messages})



> Entering new AgentExecutor chain...

Invoking: `get_current_time` with `{'location': 'Busan', 'timezone': 'Asia/Seoul'}`


Asia/Seoul (Busan) 현재시각 2025-09-13 17:57:37부산은 2025년 9월 13일 17시 57분 37초입니다.

> Finished chain.
